## 연습문제 2: 쓰레드

이 파일은 쓰레드 대한 연습문제를 다루는 노트입니다.

날짜: 2025.9.24

학번: 202110763

이름: 신연준


* 연습문제 6-9. 파이썬 언어에서 다음에 대해 조사하고 예제코드를 실행하여 보라
  - 1. 파이썬으로 싱글톤 패턴을 구현하는 방법들
  - 2. 파이썬에서 멀티쓰레드 구현하는 기법
  - 3. 멀티쓰레드에서 공유 변수에 대한 Critical Section을 보호하는 기법


In [ ]:
# 1. 싱글톤 패턴 구현 방법들

print("=== 1. 싱글톤 패턴 구현 방법들 ===")

# 방법 1: __new__ 메소드 사용
class Singleton1:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, 'initialized'):
            self.value = "싱글톤 객체"
            self.initialized = True

# 방법 2: 데코레이터 사용
def singleton(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton
class Singleton2:
    def __init__(self):
        self.value = "데코레이터 싱글톤"

# 방법 3: 메타클래스 사용
class SingletonMeta(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Singleton3(metaclass=SingletonMeta):
    def __init__(self):
        self.value = "메타클래스 싱글톤"

# 테스트
print("싱글톤 패턴 테스트:")
s1_1 = Singleton1()
s1_2 = Singleton1()
print(f"Singleton1 - 같은 객체인가? {s1_1 is s1_2}")

s2_1 = Singleton2()
s2_2 = Singleton2()
print(f"Singleton2 - 같은 객체인가? {s2_1 is s2_2}")

s3_1 = Singleton3()
s3_2 = Singleton3()
print(f"Singleton3 - 같은 객체인가? {s3_1 is s3_2}")

print("\n=== 2. 멀티쓰레드 구현 기법 ===")

import threading
import time

# 기본 쓰레드 예제
def worker(name, delay):
    for i in range(3):
        print(f"쓰레드 {name}: 작업 {i+1}")
        time.sleep(delay)

# 쓰레드 생성 및 실행
thread1 = threading.Thread(target=worker, args=("A", 0.5))
thread2 = threading.Thread(target=worker, args=("B", 0.3))

print("멀티쓰레드 실행:")
thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("\n=== 3. Critical Section 보호 기법 ===")

# 공유 변수
shared_counter = 0
lock = threading.Lock()

def increment_counter(name, iterations):
    global shared_counter
    for i in range(iterations):
        # Critical Section 보호
        with lock:
            temp = shared_counter
            time.sleep(0.001)  # 시뮬레이션을 위한 지연
            shared_counter = temp + 1
        print(f"쓰레드 {name}: 카운터 증가")

# Lock을 사용하지 않은 경우 (문제 상황)
def increment_without_lock(name, iterations):
    global shared_counter
    for i in range(iterations):
        temp = shared_counter
        time.sleep(0.001)
        shared_counter = temp + 1

print("Critical Section 보호 테스트:")
shared_counter = 0
threads = []
for i in range(3):
    t = threading.Thread(target=increment_counter, args=(f"Thread-{i+1}", 5))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f"Lock 사용 후 최종 카운터 값: {shared_counter}")

# Lock을 사용하지 않은 경우의 문제점 시연
print("\nLock을 사용하지 않은 경우의 문제점:")
shared_counter = 0
threads = []
for i in range(3):
    t = threading.Thread(target=increment_without_lock, args=(f"Thread-{i+1}", 5))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print(f"Lock 미사용 후 최종 카운터 값: {shared_counter}")
print("(예상값: 15, 실제값이 다를 수 있음 - 경쟁 상태 때문)")
